In [1]:
# Hugging Face/PEFT/bitsandbytes 등
!pip -q install "accelerate>=0.34.2" "bitsandbytes>=0.43.1" pillow pandas --upgrade
# Transformer 최신 버전 설치 (소스에서 설치 권장)
!pip install "git+https://github.com/huggingface/transformers"


# ⚠️ 핵심: CUDA 12.1용 PyTorch/torchvision 설치 (CPU 빌드 방지)
!pip -q install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu121


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 151.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-azteio71
  Running command git clone --filter

### 1) 환경 준비: GPU/라이브러리, 드라이브 마운트, 데이터 unzip

In [1]:
# ⏱️ 시작
import time, os, json, random
from datetime import datetime
t0_boot = time.time()

# ⚙️ GPU/버전 점검 + 최적 옵션
import torch, platform
print("Python:", platform.python_version())
print("PyTorch:", torch.__version__)
print("CUDA 사용 가능:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("GPU 메모리(총 MB):", torch.cuda.get_device_properties(0).total_memory // (1024**2))

# 💾 구글 드라이브 마운트 & 데이터 unzip
from google.colab import drive
drive.mount('/content/drive')

DATA_ZIP = "/content/drive/My Drive/aichallenge/data.zip"
UNZIP_DIR = "/content/"
if os.path.exists(DATA_ZIP):
    !unzip -o "/content/drive/My Drive/aichallenge/data.zip" -d "/content/"
else:
    print("⚠️ data.zip이 드라이브에 없습니다. 경로 확인 필요.")

# 🔒 재현성(과도한 시드고정은 VLM엔 큰 영향없지만 기본 세팅)
import numpy as np, random
random.seed(42); np.random.seed(42); torch.manual_seed(42)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(42)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/test/test_2776.jpg  
  inflating: /content/test/test_2777.jpg  
  inflating: /content/test/test_2778.jpg  
  inflating: /content/test/test_2779.jpg  
  inflating: /content/test/test_2780.jpg  
  inflating: /content/test/test_2781.jpg  
  inflating: /content/test/test_2782.jpg  
  inflating: /content/test/test_2783.jpg  
  inflating: /content/test/test_2784.jpg  
  inflating: /content/test/test_2785.jpg  
  inflating: /content/test/test_2786.jpg  
  inflating: /content/test/test_2787.jpg  
  inflating: /content/test/test_2788.jpg  
  inflating: /content/test/test_2789.jpg  
  inflating: /content/test/test_2790.jpg  
  inflating: /content/test/test_2791.jpg  
  inflating: /content/test/test_2792.jpg  
  inflating: /content/test/test_2793.jpg  
  inflating: /content/test/test_2794.jpg  
  inflating: /content/test/test_2795.jpg  
  inflating: /content/test/test_2796.jpg  
  inflating: /content/test/test_2797.jpg  
  inflating: /cont

### 0) 공통: 실험 루트/유틸

In [2]:


# 📁 실험 루트(날짜-시간 스탬프)
EXP_TIME = datetime.now().strftime("%Y%m%d_%H%M%S")
EXP_ROOT = f"/content/drive/My Drive/aichallenge/experiments/{EXP_TIME}"
SUBDIRS = ["logs", "preds", "analysis", "submissions"]
for d in SUBDIRS:
    os.makedirs(os.path.join(EXP_ROOT, d), exist_ok=True)

# 🧪 태그 유틸
def make_tag(model_short, image_size, prompt_style, rev=1):
    return f"{model_short}__sz{image_size}__ps{prompt_style}__rev{rev}"

print("실험 루트:", EXP_ROOT)
print("부팅시간(sec):", round(time.time()-t0_boot, 2))


실험 루트: /content/drive/My Drive/aichallenge/experiments/20251026_080951
부팅시간(sec): 49.21


### 2) 데이터 로더 & 샘플링/셔플 유틸

In [3]:
# 📚 CSV 로드 & 컬럼 자동탐지
import pandas as pd
from PIL import Image

# 기본 경로
TRAIN_CSV = "/content/train.csv"
TEST_CSV  = "/content/test.csv"
PILOT_CSV = "/content/drive/My Drive/aichallenge/pilot_300.csv"  # 제공됨 가정

def detect_columns(df):
    # 가벼운 자동탐지: id, path, question, a,b,c,d, answer
    cols = df.columns.str.lower().tolist()
    id_col = [c for c in df.columns if c.lower() in ["id","image_id"]][0]
    path_col = [c for c in df.columns if c.lower() in ["path","image_path","img_path"]][0]
    q_col = [c for c in df.columns if c.lower() in ["question","prompt","q"]][0]
    opt = {}
    for k in ["a","b","c","d"]:
        opt[k] = [c for c in df.columns if c.lower()==k][0]
    ans_col = [c for c in df.columns if c.lower() in ["answer","label","gt"]]
    ans_col = ans_col[0] if ans_col else None
    return id_col, path_col, q_col, opt, ans_col

pilot_df = pd.read_csv(PILOT_CSV)
ID_COL, IMG_COL, Q_COL, OPT_COLS, ANS_COL = detect_columns(pilot_df)
print("탐지:", {"id":ID_COL, "img":IMG_COL, "q":Q_COL, "opt":OPT_COLS, "ans":ANS_COL})

# 🔁 선택지 셔플(원복 맵 포함)
import copy, itertools, time
def shuffle_options(row, seed=None):
    # 선택지 원본
    a,b,c,d = row[OPT_COLS["a"]], row[OPT_COLS["b"]], row[OPT_COLS["c"]], row[OPT_COLS["d"]]
    items = [("a",a),("b",b),("c",c),("d",d)]
    rng = random.Random(seed) if seed is not None else random
    rng.shuffle(items)
    # 새 레이블 매핑
    new = {"a":items[0][1],"b":items[1][1],"c":items[2][1],"d":items[3][1]}
    remap = { "a": items[0][0], "b": items[1][0], "c": items[2][0], "d": items[3][0] }
    return new["a"], new["b"], new["c"], new["d"], remap


탐지: {'id': 'id', 'img': 'path', 'q': 'question', 'opt': {'a': 'a', 'b': 'b', 'c': 'c', 'd': 'd'}, 'ans': 'answer'}


### 3) 프롬프트 템플릿 (기존 + 신규 2종)
- 메모: “사실추론 과정 출력” 유도는 오히려 잡음이 되고, 이전 실험상 하락. 위 3/4는 짧고, 답만 강제.

In [4]:
# 🧩 프롬프트 스타일
# style 0: 검증된 보수형 (짧고 명확)
# style 2: "오직 한 글자" 규제형 (이전에 상승 관찰)
# style 3: 신규) "선지 재진술 + 답만"  — 모델이 보기 구조를 다시 ‘읽게’ 만들고 정답은 한 글자만
# style 4: 신규) "문장 금지 + 철자만" — 말줄임 및 부가 텍스트 강한 억제

SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)

def build_prompt(question, a,b,c,d, style:int):
    if style == 0:
        return (
            f"Question: {question}\n"
            f"a) {a}\n"
            f"b) {b}\n"
            f"c) {c}\n"
            f"d) {d}\n"
            "Answer with exactly one letter: a, b, c, or d."
        )
    if style == 2:
        return (
            f"{question}\n"
            f"Options: a) {a}; b) {b}; c) {c}; d) {d}\n"
            "Respond with one lowercase letter only (a/b/c/d). No explanation."
        )
    if style == 3:
        return (
            f"Question: {question}\n"
            f"Options are:\n"
            f"a: {a}\n"
            f"b: {b}\n"
            f"c: {c}\n"
            f"d: {d}\n"
            "Select the best option and output only its letter (a/b/c/d)."
        )
    if style == 4:
        return (
            f"{question}\n"
            f"a: {a}\nb: {b}\nc: {c}\nd: {d}\n"
            "Do not write words. Output only one of: a b c d."
        )
    # 기본 fallback = 0
    return build_prompt(question, a,b,c,d, 0)


### 4) 모델 로더 (공식 권장 방식)

In [7]:
# 🤖 Qwen VL 로더(공식 문서 스타일)
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, Qwen2_5_VLForConditionalGeneration, Qwen3VLForConditionalGeneration, AutoProcessor

def load_qwen_vl(model_id:str, torch_dtype=torch.float16):
    print("Loading:", model_id)
    if model_id == "Qwen/Qwen2.5-VL-7B-Instruct":
        model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
            model_id,
            torch_dtype="auto",
            device_map="auto",
            trust_remote_code=True,
        )

    elif model_id == "Qwen/Qwen3-VL-8B-Instruct":
        model = Qwen3VLForConditionalGeneration.from_pretrained(
            model_id,
            torch_dtype="auto",
            device_map="auto",
            trust_remote_code=True,
        )

    elif model_id == "Qwen/Qwen2-VL-7B-Instruct":
        model = Qwen2VLForConditionalGeneration.from_pretrained(
            model_id,
            torch_dtype="auto",
            device_map="auto",
            trust_remote_code=True,
        )

    processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
    # 생성은 매우 짧게 → 캐시 비활성로 메모리 여유
    model.generation_config.max_new_tokens = 2
    model.config.use_cache = False
    return model, processor


### 5) 이미지 리사이즈 정책(리소스 안전모드)

384, 512, 672, 768, 896 지원

768/896에서는 메모리 여유 확보를 위해 이미지 정사각 리사이즈(긴변 맞춤) 후 입력.

Qwen의 동적 해상도 이점은 있으나, 대회 시간/자원 제약으로 단일 리사이즈가 안전.

In [9]:
# # 🖼️ 리사이즈 유틸(정사각 리사이즈)
# def load_image_resized(path, image_size=384):
#     img = Image.open(path).convert("RGB")
#     if image_size is not None:
#         img = img.resize((image_size, image_size))
#     return img


from PIL import Image

def load_image_resized(path, image_size=384):
    """
    이미지 비율을 유지하면서 리사이즈하고,
    남는 공간은 검은색으로 패딩하여 정사각형 이미지를 반환.
    """
    img = Image.open(path).convert("RGB")

    # 1. 원본 비율을 유지하면서 image_size에 맞게 리사이즈
    # (긴 변이 image_size에 맞도록 축소)
    img.thumbnail((image_size, image_size), Image.Resampling.LANCZOS)

    # 2. image_size 크기의 검은색 정사각형 캔버스 생성
    padded_img = Image.new("RGB", (image_size, image_size), (0, 0, 0))

    # 3. 리사이즈된 이미지를 캔버스 중앙에 붙여넣기
    paste_x = (image_size - img.width) // 2
    paste_y = (image_size - img.height) // 2
    padded_img.paste(img, (paste_x, paste_y))

    return padded_img

# --- 예시 사용 ---
# img = load_image_resized_with_padding("내_이미지.jpg", 384)
# img.show() # 결과 확인

### 6) 단일 설정 파일럿 평가(50/300 선택), 오답 즉시 로그 & CSV 저장

In [22]:
import math
import re
from tqdm import tqdm

# ✅ 모델 응답에서 선지(a/b/c/d)만 추출
def extract_choice(text: str) -> str:
    t = text.strip().lower()
    # 마지막 줄/토큰에서 우선 탐색
    lines = [l.strip() for l in t.splitlines() if l.strip()]
    last = lines[-1] if lines else t
    if last in ["a","b","c","d"]:
        return last
    for tok in re.findall(r"[a-d]", last):
        return tok
    return "a"  # 안전장치

def quick_eval(model, processor, df, image_size=384, prompt_style=0,
               max_samples=50, log_every=10, tag=None, save_dir=None, do_shuffle=False):
    t0_total = time.time()
    preds, is_corrects, lat_ms = [], [], []
    wrong_rows = []
    n = min(len(df), max_samples)
    for i in tqdm(range(n), desc=f"pilot sz={image_size}, ps={prompt_style}", unit="sample"):
        row = df.iloc[i]
        a,b,c,d, remap = shuffle_options(row, seed=i) if do_shuffle else (
            row[OPT_COLS["a"]], row[OPT_COLS["b"]], row[OPT_COLS["c"]],
            row[OPT_COLS["d"]], {"a":"a","b":"b","c":"c","d":"d"}
        )
        user_text = build_prompt(row[Q_COL], a,b,c,d, prompt_style)
        img = load_image_resized(row[IMG_COL], image_size)

        messages = [
            {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
            {"role": "user",   "content": [
                {"type": "image", "image": img},
                {"type": "text",  "text": user_text},
            ]},
        ]

        # ⏱️ 개별
        t0 = time.time()
        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=2,
                do_sample=False,
                temperature=0.0,
                eos_token_id=processor.tokenizer.eos_token_id,
            )
        text = processor.batch_decode(out, skip_special_tokens=True)[0]
        elapsed = (time.time()-t0)*1000

        choice = extract_choice(text)
        # 원 레이블 복원
        choice = remap.get(choice, choice)
        preds.append(choice)
        lat_ms.append(elapsed)

        if ANS_COL:
            gt = str(row[ANS_COL]).strip().lower()
            is_ok = int(choice == gt)
            is_corrects.append(is_ok)
            if is_ok==0:
                wrong_rows.append({
                    "id": row[ID_COL], "pred": choice, "gt": gt,
                    "question": row[Q_COL],
                    "a": row[OPT_COLS["a"]], "b": row[OPT_COLS["b"]],
                    "c": row[OPT_COLS["c"]], "d": row[OPT_COLS["d"]],
                    "raw": text
                })
        else:
            is_corrects.append(None)

        # 🔔 10개 단위 진행 요약 + 오답 즉시
        if (i+1) % log_every == 0 and ANS_COL:
            acc = sum(is_corrects)/len(is_corrects)
            print(f"[{i+1}개] 누적 정확도={acc:.4f}, 평균지연={np.mean(lat_ms):.1f}ms")
        if ANS_COL and is_corrects[-1]==0:
            print(f"❌ 오답 id={row[ID_COL]} | pred={choice}, gt={gt} | Q={row[Q_COL]}")

    # 📊 요약
    acc = sum(is_corrects)/len(is_corrects) if ANS_COL else None
    print(f"✅ 최종 정확도={acc:.4f}" if acc is not None else "✅ 평가(정답 없음) 완료",
          f", 총소요={time.time()-t0_total:.2f}s, 평균지연={np.mean(lat_ms):.1f}ms")

    # 💾 저장
    if tag and save_dir:
        out_path = os.path.join(save_dir, f"pilot__{tag}.csv")
        rows = []
        for i in range(n):
            rid = df.iloc[i][ID_COL]
            gt = str(df.iloc[i][ANS_COL]).strip().lower() if ANS_COL else ""
            rows.append({
                "id": rid, "pred": preds[i], "answer": gt,
                "is_correct": (preds[i]==gt) if ANS_COL else "",
                "latency_ms": round(lat_ms[i],1), "tag": tag
            })
        pd.DataFrame(rows).to_csv(out_path, index=False)
        # 오답 로그
        if wrong_rows:
            pd.DataFrame(wrong_rows).to_csv(os.path.join(EXP_ROOT,"logs", f"wrongs__{tag}.csv"), index=False)
        print("📁 저장:", out_path)

    return acc, preds


### 7) 파일럿 실행 드라이버 (안전 해상도 세트 포함)

권장 흐름: Qwen3 → Qwen2.5까지만 우선 평가. 결과를 보고 896 유지/제외 결정.
4bit(Qwen2)는 오답 다양성이 충분히 다르면만 후보로 포함.

In [23]:
# ✅ 모델 하나씩 로드해서 순차 평가(메모리 절약)
CAND_MODELS = [
    ("Qwen/Qwen3-VL-8B-Instruct",  "qwen3_8b"),
    ("Qwen/Qwen2.5-VL-7B-Instruct","qwen25_7b"),
    ("Qwen/Qwen2-VL-7B-Instruct",  "qwen2_7b")
]

# ⚠️ 리소스-세이프 해상도(672 초과 시험치 포함)
SAFE_SIZES = [384, 512, 672, 768, 896]  # 768/896는 시간 증가 가능성 있음
STYLES_TRY  = [0, 2, 3, 4]               # 1은 과거 저하 → 기본 제외

# 📊 먼저 pilot_50으로 빠른 컷
PILOT_N = 50    # 필요시 300으로 늘려 재검증

for model_id, short in CAND_MODELS:

    model, proc = load_qwen_vl(model_id)
    for sz in SAFE_SIZES:
        for st in STYLES_TRY:
            tag = make_tag(short, sz, st, rev=1)
            print(f"\n🚀 시험: {tag}")
            acc, _ = quick_eval(
                model, proc, pilot_df,
                image_size=sz, prompt_style=st,
                max_samples=PILOT_N, log_every=10,
                tag=tag, save_dir=os.path.join(EXP_ROOT, "preds"),
                do_shuffle=False  # 기본 고정 보기(셔플은 별도 실험)
            )
    # 메모리 해제
    del model; torch.cuda.empty_cache()


Loading: Qwen/Qwen3-VL-8B-Instruct


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


🚀 시험: qwen3_8b__sz384__ps0__rev1


pilot sz=384, ps=0:  20%|██        | 10/50 [00:32<02:11,  3.29s/sample]

...

[40개] 누적 정확도=0.8750, 평균지연=239.1ms


pilot sz=896, ps=4:  96%|█████████▌| 48/50 [00:11<00:00,  4.12sample/s]

❌ 오답 id=train_3297 | pred=b, gt=d | Q=이 이미지에서 유리 그릇에 담긴 노란색 음식은 무엇일까요?


pilot sz=896, ps=4: 100%|██████████| 50/50 [00:12<00:00,  4.10sample/s]

[50개] 누적 정확도=0.8800, 평균지연=238.9ms
✅ 최종 정확도=0.8800 , 총소요=12.21s, 평균지연=238.9ms
📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/pilot__qwen2_7b__sz896__ps4__rev1.csv


### 8) 비교/다양성 분석(정확도표 + 오답 Jaccard)

In [24]:
import glob
import numpy as np

def collect_pilot_results(pred_dir):
    paths = glob.glob(os.path.join(pred_dir, "pilot__*.csv"))
    rows = []
    for p in paths:
        df = pd.read_csv(p)
        tag = os.path.basename(p).replace("pilot__","").replace(".csv","")
        acc = df["is_correct"].mean()
        lat = df["latency_ms"].mean()
        rows.append({"tag":tag, "acc":acc, "avg_latency_ms":lat, "n":len(df)})
    res = pd.DataFrame(rows).sort_values(["acc","avg_latency_ms"], ascending=[False, True])
    res.to_csv(os.path.join(EXP_ROOT,"analysis","pilot_compare.csv"), index=False)
    return res

def jaccard_wrong(pred_dir, top_k=6):
    res = collect_pilot_results(pred_dir)
    res = res.head(top_k)  # 상위 몇 개만 상호분석
    tags = res["tag"].tolist()
    wrong_sets = {}
    for t in tags:
        df = pd.read_csv(os.path.join(pred_dir, f"pilot__{t}.csv"))
        wrong = set(df[df["is_correct"]==0]["id"].tolist())
        wrong_sets[t] = wrong
    # Jaccard 행렬
    mat = np.zeros((len(tags), len(tags)))
    for i, ti in enumerate(tags):
        for j, tj in enumerate(tags):
            a, b = wrong_sets[ti], wrong_sets[tj]
            inter = len(a & b); union = len(a | b) if len(a|b)>0 else 1
            mat[i,j] = inter/union
    mat_df = pd.DataFrame(mat, index=tags, columns=tags)
    mat_df.to_csv(os.path.join(EXP_ROOT,"analysis","pilot_diversity.csv"))
    # 간단 보완성 스코어
    res["div_score"] = 0.0
    base = mat.mean()
    return res, mat_df

cmp_df, jac_df = jaccard_wrong(os.path.join(EXP_ROOT,"preds"))
print("📁 저장:", os.path.join(EXP_ROOT,"analysis","pilot_compare.csv"))
print("📁 저장:", os.path.join(EXP_ROOT,"analysis","pilot_diversity.csv"))
display(cmp_df.head(10))
display(jac_df)


📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/analysis/pilot_compare.csv
📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/analysis/pilot_diversity.csv


,tag,acc,avg_latency_ms,n,div_score
53,qwen2_7b__sz768__ps2__rev1,0.96,183.660,50,0.0
11,qwen3_8b__sz672__ps4__rev1,0.96,2898.358,50,0.0
9,qwen3_8b__sz672__ps2__rev1,0.96,2899.388,50,0.0
8,qwen3_8b__sz672__ps0__rev1,0.96,2912.122,50,0.0
50,qwen2_7b__sz672__ps3__rev1,0.94,156.978,50,0.0
51,qwen2_7b__sz672__ps4__rev1,0.94,158.248,50,0.0


,qwen2_7b__sz768__ps2__rev1,qwen3_8b__sz672__ps4__rev1,qwen3_8b__sz672__ps2__rev1,qwen3_8b__sz672__ps0__rev1,qwen2_7b__sz672__ps3__rev1,qwen2_7b__sz672__ps4__rev1
qwen2_7b__sz768__ps2__rev1,1.000000,0.333333,0.000000,0.000000,0.25,0.25
qwen3_8b__sz672__ps4__rev1,0.333333,1.000000,0.333333,0.333333,0.00,0.00
qwen3_8b__sz672__ps2__rev1,0.000000,0.333333,1.000000,0.333333,0.25,0.25
qwen3_8b__sz672__ps0__rev1,0.000000,0.333333,0.333333,1.000000,0.00,0.00
qwen2_7b__sz672__ps3__rev1,0.250000,0.000000,0.250000,0.000000,1.00,1.00
qwen2_7b__sz672__ps4__rev1,0.250000,0.000000,0.250000,0.000000,1.00,1.00


### 9) (선택) 가중치 산정

In [25]:
def weights_from_accuracy(tags):
    comp = pd.read_csv(os.path.join(EXP_ROOT, "analysis", "pilot_compare.csv"))
    sub = comp[comp["tag"].isin(tags)].copy()
    if sub.empty or sub["acc"].sum() == 0:
        # 가중치 불가 → 균등 가중치
        return {t: 1.0/len(tags) for t in tags}
    sub["w"] = sub["acc"] / sub["acc"].sum()
    return dict(zip(sub["tag"], sub["w"]))

# 예) 상위 3개 선정 후 가중치
# chosen = cmp_df.head(3)["tag"].tolist()
# W = weights_from_accuracy(chosen)
# print("가중치:", W)


### 10) 파일럿 앙상블 검증(다수결→가중치 타이브레이크)

In [33]:

from collections import Counter
import hashlib

def ensemble_pilot(tags, use_weights=True):
    # id 기준 머지
    base = None
    for t in tags:
        df = pd.read_csv(os.path.join(EXP_ROOT, "preds", f"pilot__{t}.csv"))[["id", "pred", "answer"]]
        df = df.rename(columns={"pred": t})
        base = df if base is None else base.merge(df[["id", t]], on="id", how="inner")

    W = weights_from_accuracy(tags) if use_weights else {t:1.0 for t in tags}

    finals, corrects = [], []
    for _, row in base.iterrows():
        votes = {t: row[t] for t in tags}
        cnt = Counter(votes.values())
        top = cnt.most_common()
        if len(top) == 1 or top[0][1] > top[1][1]:
            final = top[0][0]
        else:
            cand = [k for k, v in cnt.items() if v == top[0][1]]
            sums = {c: sum(W[t] for t in tags if votes[t] == c) for c in cand}
            final = max(sums.items(), key=lambda x: x[1])[0]
        finals.append(final)
        corrects.append(int(final == row["answer"]))

    tag_hash = hashlib.md5(("|".join(tags)).encode()).hexdigest()[:8]
    out = base[["id", "answer"]].copy()
    out["final_pred"] = finals
    out["is_correct"] = corrects
    out.to_csv(os.path.join(EXP_ROOT, "analysis", f"pilot_ensemble__{tag_hash}.csv"), index=False)

    acc = sum(corrects) / len(corrects)
    print(f"🧪 앙상블 후보={len(tags)}개 | 파일럿 정확도={acc:.4f}")
    return acc, tag_hash



# def ensemble_pilot(tags, use_weights=True):
#     # 입력 CSV 합치기
#     dfs = [pd.read_csv(os.path.join(EXP_ROOT,"preds", f"pilot__{t}.csv")) for t in tags]
#     base = dfs[0][["id","answer"]].copy()
#     for i,t in enumerate(tags):
#         base[t] = dfs[i]["pred"]

#     W = weights_from_accuracy(tags) if use_weights else {t:1.0 for t in tags}

#     finals, corrects = [], []
#     for _, row in base.iterrows():
#         votes = {t:row[t] for t in tags}
#         # 다수결
#         cnt = Counter(votes.values())
#         top = cnt.most_common()
#         if len(top) == 1 or top[0][1] > top[1][1]:
#             final = top[0][0]
#         else:
#             # 동률이면 가중치 합 최대인 후보
#             cand = [k for k,v in cnt.items() if v==top[0][1]]
#             sums = {c: sum(W[t] for t in tags if votes[t]==c) for c in cand}
#             final = max(sums.items(), key=lambda x:x[1])[0]
#         finals.append(final)
#         corrects.append(int(final==row["answer"]))

#     tag_hash = hashlib.md5(("|".join(tags)).encode()).hexdigest()[:8]
#     out = base[["id","answer"]].copy()
#     out["final_pred"] = finals
#     out["is_correct"] = corrects
#     out.to_csv(os.path.join(EXP_ROOT,"analysis", f"pilot_ensemble__{tag_hash}.csv"), index=False)

#     acc = sum(corrects)/len(corrects)
#     print(f"🧪 앙상블 후보={len(tags)}개 | 파일럿 정확도={acc:.4f}")
#     return acc, tag_hash

# 예) 상위 3개 태그 골라 실행

# top3 = cmp_df.head(3)["tag"].tolist()
top3 = ['qwen2_7b__sz768__ps2__rev1', 'qwen3_8b__sz672__ps4__rev1', 'qwen3_8b__sz672__ps0__rev1']
print(top3)
ensemble_pilot(top3, use_weights=True)


['qwen2_7b__sz768__ps2__rev1', 'qwen3_8b__sz672__ps4__rev1', 'qwen3_8b__sz672__ps0__rev1']
🧪 앙상블 후보=3개 | 파일럿 정확도=0.9600


(0.96, '9dc522b2')

### 11) 테스트셋 단일 추론 저장(선택된 태그만 순차 실행)

In [34]:
def infer_test_and_save(model, processor, test_csv, image_size, prompt_style, tag):
    t0_tot = time.time()
    test_df = pd.read_csv(test_csv)
    preds = []


    for i in tqdm(range(len(test_df)), desc=f"test {tag}", unit="sample"):
        row = test_df.iloc[i]
        img = load_image_resized(row[IMG_COL], image_size)
        user_text = build_prompt(row[Q_COL], row[OPT_COLS["a"]], row[OPT_COLS["b"]],
                                 row[OPT_COLS["c"]], row[OPT_COLS["d"]], prompt_style)
        messages = [
            {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
            {"role": "user",   "content": [
                {"type": "image", "image": img},
                {"type": "text",  "text": user_text},
            ]},
        ]

        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}


        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=2,
                do_sample=False,
                temperature=0.0,
                eos_token_id=processor.tokenizer.eos_token_id,
            )
        text = processor.batch_decode(out, skip_special_tokens=True)[0]
        preds.append(extract_choice(text))
    df_out = pd.DataFrame({"id": test_df[ID_COL], "pred": preds, "tag": tag})
    out_path = os.path.join(EXP_ROOT,"preds", f"test__{tag}.csv")
    df_out.to_csv(out_path, index=False)
    print("📁 저장:", out_path, "| 총소요(s):", round(time.time()-t0_tot,2))


In [36]:
# Qwen3 8B, ps4@672
model, proc = load_qwen_vl("Qwen/Qwen3-VL-8B-Instruct")
infer_test_and_save(model, proc, TEST_CSV, 672, 4, "qwen3_8b__sz672__ps4__rev1")
del model; torch.cuda.empty_cache()

Loading: Qwen/Qwen3-VL-8B-Instruct


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

test qwen3_8b__sz672__ps4__rev1: 100%|██████████| 3887/3887 [11:53<00:00,  5.45sample/s]


📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test__qwen3_8b__sz672__ps4__rev1.csv | 총소요(s): 713.43


In [37]:
# Qwen3 8B, ps0@672
model, proc = load_qwen_vl("Qwen/Qwen3-VL-8B-Instruct")
infer_test_and_save(model, proc, TEST_CSV, 672, 0, "qwen3_8b__sz672__ps0__rev1")
del model; torch.cuda.empty_cache()

Loading: Qwen/Qwen3-VL-8B-Instruct


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

test qwen3_8b__sz672__ps0__rev1: 100%|██████████| 3887/3887 [11:49<00:00,  5.48sample/s]


📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test__qwen3_8b__sz672__ps0__rev1.csv | 총소요(s): 709.38


In [38]:
# Qwen2 7B, ps2@768
model, proc = load_qwen_vl("Qwen/Qwen2-VL-7B-Instruct")
infer_test_and_save(model, proc, TEST_CSV, 768, 2, "qwen2_7b__sz768__ps2__rev1")
del model; torch.cuda.empty_cache()

Loading: Qwen/Qwen2-VL-7B-Instruct


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

test qwen2_7b__sz768__ps2__rev1: 100%|██████████| 3887/3887 [12:19<00:00,  5.25sample/s]


📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test__qwen2_7b__sz768__ps2__rev1.csv | 총소요(s): 739.72


### 12) 테스트셋 앙상블 & 제출 파일 생성

In [43]:
from collections import Counter

def build_submission_from_tests(tags, sample_csv=TEST_CSV, out_name=None, prefer="qwen3_8b"):
    test_df = pd.read_csv(sample_csv)
    base = pd.DataFrame({"id": test_df[ID_COL]})
    for t in tags:
        df = pd.read_csv(os.path.join(EXP_ROOT,"preds", f"test__{t}.csv"))
        base[t] = df["pred"]

    # 다수결 → 동률은 (가중치>선호모델) 순
    W = weights_from_accuracy(tags)
    finals = []
    for _, row in base.iterrows():
        votes = {t:row[t] for t in tags}
        cnt = Counter(votes.values())
        top = cnt.most_common()
        if len(top)==1 or top[0][1] > top[1][1]:
            final = top[0][0]
        else:
            cand = [k for k,v in cnt.items() if v==top[0][1]]
            sums = {c: sum(W.get(t,1.0) for t in tags if votes[t]==c) for c in cand}
            best = max(sums.items(), key=lambda x:x[1])[0]
            if list(sums.values()).count(sums[best])>1:
                # 여전히 동률이면 선호모델 출력 우선
                finals.append(votes[prefer])
                continue
            final = best
        finals.append(final)

    sub = pd.DataFrame({"id": base["id"], "answer": finals})
    out = out_name or f"submission__{datetime.now().strftime('%m%d_%H%M')}.csv"
    out_path = os.path.join(EXP_ROOT,"submissions", out)
    sub.to_csv(out_path, index=False)
    print("📁 최종 제출 파일:", out_path)
    return out_path


In [45]:
build_submission_from_tests(top3, sample_csv=TEST_CSV, prefer='qwen3_8b__sz672__ps4__rev1')

📁 최종 제출 파일: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission__1026_1106.csv


'/content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission__1026_1106.csv'

### 7-1) 파일럿 300 샘플 재평가 (선택 조합)

In [47]:
# 파일럿 300 샘플로 재평가
PILOT_N_FULL = 300

# 50개 파일럿 결과에서 정확도 0.94 이상인 조합 선택
# cmp_df가 로드되지 않았을 경우를 대비하여 확인
if 'cmp_df' not in locals():
    print("⚠️ cmp_df가 로드되지 않았습니다. 셀 8)을 먼저 실행하여 결과를 생성해주세요.")
else:
    # 정확도 0.94 이상인 조합 필터링
    selected_combos_df = cmp_df[cmp_df["acc"] >= 0.94].copy()

    # (모델_ID, 해상도, 프롬프트_스타일) 튜플 리스트 생성
    # short 이름을 model_id로 변환해야 함
    selected_combos_for_pilot300 = []
    for index, row in selected_combos_df.iterrows():
        tag = row["tag"]
        parts = tag.split("__")
        short_name = parts[0]
        sz = int(parts[1].replace("sz", ""))
        st = int(parts[2].replace("ps", ""))
        # short 이름에 해당하는 model_id 찾기
        model_id = next((m_id for m_id, sh in CAND_MODELS if sh == short_name), None)
        if model_id:
            selected_combos_for_pilot300.append((model_id, sz, st))
        else:
            print(f"⚠️ short 이름 {short_name}에 해당하는 모델 ID를 찾을 수 없습니다. 건너뜀.")


    if not selected_combos_for_pilot300:
        print("선택된 조합이 없습니다. 정확도 기준을 확인해주세요.")
    else:
        print(f"▶️ 파일럿 300 샘플 재평가 시작 (총 {len(selected_combos_for_pilot300)} 조합)")
        # 기존 pilot_df (300개 샘플) 사용
        if 'pilot_df' not in locals():
            print("⚠️ pilot_df가 로드되지 않았습니다. 셀 2)를 먼저 실행해주세요.")
        else:
            for model_id, sz, st in selected_combos_for_pilot300:
                # 모델 로드
                model, proc = load_qwen_vl(model_id)

                # short 이름 찾기 (CAND_MODELS 리스트는 그대로 사용)
                short_name = next(sh for m_id, sh in CAND_MODELS if m_id == model_id)


                tag = make_tag(short_name, sz, st, rev=1)
                print(f"\n🚀 시험: {tag} (PILOT_N={PILOT_N_FULL})")

                # quick_eval 함수 호출 (300개 샘플)
                acc, _ = quick_eval(
                    model, proc, pilot_df,
                    image_size=sz,
                    prompt_style=st,
                    max_samples=PILOT_N_FULL,
                    log_every=50, # 300개니까 로그 간격 조정
                    tag=tag,
                    save_dir=os.path.join(EXP_ROOT, "preds"),
                    do_shuffle=False
                )

                # 메모리 해제
                del model
                torch.cuda.empty_cache()

            print("\n✅ 파일럿 300 샘플 재평가 완료.")

▶️ 파일럿 300 샘플 재평가 시작 (총 6 조합)
Loading: Qwen/Qwen2-VL-7B-Instruct


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]


🚀 시험: qwen2_7b__sz768__ps2__rev1 (PILOT_N=300)


pilot sz=768, ps=2:   9%|▊         | 26/300 [00:04<00:51,  5.34sample/s]

...

❌ 오답 id=train_3579 | pred=a, gt=d | Q=이 한식 상차림에서 보이는 주된 반찬은 무엇인가요?


pilot sz=672, ps=4:  91%|█████████▏| 274/300 [00:48<00:04,  5.72sample/s]

❌ 오답 id=train_1876 | pred=d, gt=a | Q=이 사진에 보이는 거대한 조형물은 무엇을 형상화한 것일까요?


pilot sz=672, ps=4: 100%|██████████| 300/300 [00:52<00:00,  5.69sample/s]


[300개] 누적 정확도=0.9167, 평균지연=159.8ms
✅ 최종 정확도=0.9167 , 총소요=52.69s, 평균지연=159.8ms
📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/pilot__qwen2_7b__sz672__ps4__rev1.csv

✅ 파일럿 300 샘플 재평가 완료.


### 8-1) 비교/다양성 분석 (파일럿 300 샘플 결과)

In [52]:
# 300개 파일럿 샘플 결과에 대한 비교/다양성 분석
print("▶️ 파일럿 300 샘플 결과 분석 시작")

# 결과 수집 및 비교표 생성
# pilot_compare.csv와 pilot_diversity.csv는 EXP_ROOT/analysis에 저장됩니다.
# 이전에 50개 파일럿 결과로 생성된 파일에 덮어쓰게 됩니다.
cmp_df_300, jac_df_300 = jaccard_wrong(os.path.join(EXP_ROOT,"preds"))

print("📁 저장:", os.path.join(EXP_ROOT,"analysis","pilot_compare.csv"))
print("📁 저장:", os.path.join(EXP_ROOT,"analysis","pilot_diversity.csv"))

print("\n📊 파일럿 300 샘플 결과 비교 (n=300)")
# n=300인 결과만 필터링하여 표시
display(cmp_df_300[cmp_df_300['n'] == 300].head(10))

print("\n🧬 파일럿 300 샘플 오답 자카드 유사도 (상위 6개 조합)")
display(jac_df_300)

print("\n✅ 파일럿 300 샘플 결과 분석 완료.")

▶️ 파일럿 300 샘플 결과 분석 시작
📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/analysis/pilot_compare.csv
📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/analysis/pilot_diversity.csv

📊 파일럿 300 샘플 결과 비교 (n=300)


,tag,acc,avg_latency_ms,n,div_score
0,qwen3_8b__sz672__ps0__rev1,0.943333,167.765000,300,0.0
1,qwen3_8b__sz672__ps2__rev1,0.940000,166.318000,300,0.0
2,qwen3_8b__sz672__ps4__rev1,0.936667,166.061667,300,0.0
5,qwen2_7b__sz768__ps2__rev1,0.923333,183.720000,300,0.0
3,qwen2_7b__sz672__ps3__rev1,0.916667,159.512333,300,0.0
4,qwen2_7b__sz672__ps4__rev1,0.916667,159.776333,300,0.0



🧬 파일럿 300 샘플 오답 자카드 유사도 (상위 6개 조합)


,qwen3_8b__sz672__ps0__rev1,qwen3_8b__sz672__ps2__rev1,qwen3_8b__sz672__ps4__rev1,qwen2_7b__sz768__ps2__rev1,qwen2_7b__sz672__ps3__rev1,qwen2_7b__sz672__ps4__rev1
qwen3_8b__sz672__ps0__rev1,1.000000,0.590909,0.714286,0.290323,0.235294,0.235294
qwen3_8b__sz672__ps2__rev1,0.590909,1.000000,0.608696,0.322581,0.303030,0.303030
qwen3_8b__sz672__ps4__rev1,0.714286,0.608696,1.000000,0.448276,0.294118,0.294118
qwen2_7b__sz768__ps2__rev1,0.290323,0.322581,0.448276,1.000000,0.548387,0.548387
qwen2_7b__sz672__ps3__rev1,0.235294,0.303030,0.294118,0.548387,1.000000,0.785714
qwen2_7b__sz672__ps4__rev1,0.235294,0.303030,0.294118,0.548387,0.785714,1.000000



✅ 파일럿 300 샘플 결과 분석 완료.


### 10-1) 파일럿 앙상블 검증 (파일럿 300 샘플 결과 기반)

In [61]:
# 파일럿 300 결과 기반으로 앙상블할 조합 선택 및 평가

# 기존 조합 유지
ensemble_tags_300 = [
    'qwen3_8b__sz672__ps0__rev1',
    'qwen3_8b__sz672__ps2__rev1',
    'qwen2_7b__sz768__ps2__rev1' # 이전 예시 조합
]

# 여러 가능성 있는 앙상블 후보 조합들 정의
ensemble_cand_2 = [
    'qwen3_8b__sz672__ps0__rev1',
    'qwen2_7b__sz768__ps2__rev1' # Qwen3 최고 + Qwen2 최고
]
ensemble_cand_3 = ensemble_tags_300 # 이전 3개 조합
ensemble_cand_4 = [
    'qwen3_8b__sz672__ps0__rev1',
    'qwen3_8b__sz672__ps2__rev1',
    'qwen3_8b__sz672__ps4__rev1', # Qwen3 672 해상도 3개 스타일
    'qwen2_7b__sz768__ps2__rev1' # + Qwen2 최고
]
ensemble_cand_5 = [
    'qwen3_8b__sz672__ps0__rev1',
    'qwen3_8b__sz672__ps2__rev1',
    'qwen3_8b__sz672__ps4__rev1',
    'qwen2_7b__sz672__ps3__rev1',
    'qwen2_7b__sz672__ps4__rev1',
    'qwen2_7b__sz768__ps2__rev1' # 파일럿 300 0.9167 이상 전부
]

# 아래는 지피티 추천조합
ensemble_cand_6_chatgpt_recommend = [
    'qwen3_8b__sz672__ps4__rev1',
    'qwen3_8b__sz672__ps2__rev1',
    'qwen2_7b__sz768__ps2__rev1'
]

# 아래는 제미나이 추천조합
ensemble_cand_7_gemini_recommend = [
    'qwen3_8b__sz672__ps0__rev1',
    'qwen2_7b__sz672__ps3__rev1'
]

ensemble_cand_8_gemini_recommend = [
    'qwen3_8b__sz672__ps0__rev1',
    'qwen2_7b__sz768__ps2__rev1',
    'qwen2_7b__sz672__ps3__rev1'
]


# 테스트할 조합들을 리스트에 담아 순회
ensemble_combinations_to_test = {
    "ensemble_cand_2": ensemble_cand_2,
    "ensemble_cand_3": ensemble_cand_3,
    "ensemble_cand_4": ensemble_cand_4,
    "ensemble_cand_5": ensemble_cand_5,
    'ensemble_cand_6_chatgpt_recommend': ensemble_cand_6_chatgpt_recommend,
    'ensemble_cand_7_gemini_recommend': ensemble_cand_7_gemini_recommend,
    'ensemble_cand_8_gemini_recommend': ensemble_cand_8_gemini_recommend
    # 추가적인 조합을 여기에 추가할 수 있습니다.

}

for combo_name, tags_to_test in ensemble_combinations_to_test.items():
    print(f"\n--- 🧪 앙상블 조합: {combo_name} ({len(tags_to_test)} 모델) ---")
    print(f"▶️ 조합 태그: {tags_to_test}")

    # ensemble_pilot 함수 호출 (파일럿 300 결과 사용)
    acc_ensemble_300, tag_hash_300 = ensemble_pilot(tags_to_test, use_weights=True)

    print(f"✅ 파일럿 300 샘플 기반 앙상블 정확도: {acc_ensemble_300:.4f}")
    print(f"📁 앙상블 결과 저장 경로: {os.path.join(EXP_ROOT, 'analysis', f'pilot_ensemble__{tag_hash_300}.csv')}")


--- 🧪 앙상블 조합: ensemble_cand_2 (2 모델) ---
▶️ 조합 태그: ['qwen3_8b__sz672__ps0__rev1', 'qwen2_7b__sz768__ps2__rev1']
🧪 앙상블 후보=2개 | 파일럿 정확도=0.9433
✅ 파일럿 300 샘플 기반 앙상블 정확도: 0.9433
📁 앙상블 결과 저장 경로: /content/drive/My Drive/aichallenge/experiments/20251026_080951/analysis/pilot_ensemble__4c7ca402.csv

--- 🧪 앙상블 조합: ensemble_cand_3 (3 모델) ---
▶️ 조합 태그: ['qwen3_8b__sz672__ps0__rev1', 'qwen3_8b__sz672__ps2__rev1', 'qwen2_7b__sz768__ps2__rev1']
🧪 앙상블 후보=3개 | 파일럿 정확도=0.9533
✅ 파일럿 300 샘플 기반 앙상블 정확도: 0.9533
📁 앙상블 결과 저장 경로: /content/drive/My Drive/aichallenge/experiments/20251026_080951/analysis/pilot_ensemble__84a9f42d.csv

--- 🧪 앙상블 조합: ensemble_cand_4 (4 모델) ---
▶️ 조합 태그: ['qwen3_8b__sz672__ps0__rev1', 'qwen3_8b__sz672__ps2__rev1', 'qwen3_8b__sz672__ps4__rev1', 'qwen2_7b__sz768__ps2__rev1']
🧪 앙상블 후보=4개 | 파일럿 정확도=0.9467
✅ 파일럿 300 샘플 기반 앙상블 정확도: 0.9467
📁 앙상블 결과 저장 경로: /content/drive/My Drive/aichallenge/experiments/20251026_080951/analysis/pilot_ensemble__a8cf0cff.csv

--- 🧪 앙상블 조합: ensemble_cand_5 (6 

### 11-1) 테스트셋 단일 추론 저장 (앙상블 후보 전체, 중복 건너뛰기)

In [59]:
# ensemble_cand_5에 정의된 모든 모델에 대해 테스트셋 단일 추론 진행 (중복 건너뛰기)

# ensemble_cand_5 조합 가져오기 (셀 10-1에서 정의됨)
if 'ensemble_cand_5' not in locals():
    print("⚠️ ensemble_cand_5 변수가 정의되지 않았습니다. 셀 10-1)을 먼저 실행해주세요.")
    tags_to_infer = [] # 빈 리스트로 설정하여 아래 루프 건너뛰기
else:
    tags_to_infer = ensemble_cand_5
    print(f"▶️ 테스트셋 단일 추론 시작 (대상 조합 수: {len(tags_to_infer)})")


for tag_to_infer in tags_to_infer:
    out_path = os.path.join(EXP_ROOT,"preds", f"test__{tag_to_infer}.csv")

    if os.path.exists(out_path):
        print(f"✅ 파일 존재: {out_path} -> 추론 건너뛰기")
        continue

    print(f"\n🚀 테스트셋 추론: {tag_to_infer}")

    # tag에서 모델 ID, 해상도, 프롬프트 스타일 추출
    parts = tag_to_infer.split("__")
    short_name = parts[0]
    sz = int(parts[1].replace("sz", ""))
    st = int(parts[2].replace("ps", ""))

    # short 이름에 해당하는 model_id 찾기
    model_id = next((m_id for m_id, sh in CAND_MODELS if sh == short_name), None)

    if model_id:
        # 모델 로드
        model, proc = load_qwen_vl(model_id)

        # infer_test_and_save 함수 호출
        infer_test_and_save(model, proc, TEST_CSV,
                            image_size=sz, prompt_style=st,
                            tag=tag_to_infer)

        # 메모리 해제
        del model
        torch.cuda.empty_cache()
    else:
        print(f"⚠️ short 이름 {short_name}에 해당하는 모델 ID 찾을 수 없음. 추론 건너뛰기.")

print("\n✅ 테스트셋 단일 추론 과정 완료.")

▶️ 테스트셋 단일 추론 시작 (대상 조합 수: 6)
✅ 파일 존재: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test__qwen3_8b__sz672__ps0__rev1.csv -> 추론 건너뛰기

🚀 테스트셋 추론: qwen3_8b__sz672__ps2__rev1
Loading: Qwen/Qwen3-VL-8B-Instruct


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

test qwen3_8b__sz672__ps2__rev1: 100%|██████████| 3887/3887 [11:50<00:00,  5.47sample/s]


📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test__qwen3_8b__sz672__ps2__rev1.csv | 총소요(s): 710.4
✅ 파일 존재: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test__qwen3_8b__sz672__ps4__rev1.csv -> 추론 건너뛰기

🚀 테스트셋 추론: qwen2_7b__sz672__ps3__rev1
Loading: Qwen/Qwen2-VL-7B-Instruct


...

📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test__qwen2_7b__sz672__ps3__rev1.csv | 총소요(s): 683.92

🚀 테스트셋 추론: qwen2_7b__sz672__ps4__rev1
Loading: Qwen/Qwen2-VL-7B-Instruct


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

test qwen2_7b__sz672__ps4__rev1: 100%|██████████| 3887/3887 [11:20<00:00,  5.71sample/s]


📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test__qwen2_7b__sz672__ps4__rev1.csv | 총소요(s): 680.95
✅ 파일 존재: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test__qwen2_7b__sz768__ps2__rev1.csv -> 추론 건너뛰기

✅ 테스트셋 단일 추론 과정 완료.


### 12-1) 테스트셋 앙상블 & 최종 제출 파일 생성

In [60]:
# ensemble_cand_5에 정의된 모델들을 사용하여 테스트셋 앙상블 및 제출 파일 생성

# ensemble_cand_5 조합 가져오기 (셀 10-1에서 정의됨)
if 'ensemble_cand_5' not in locals():
    print("⚠️ ensemble_cand_5 변수가 정의되지 않았습니다. 셀 10-1)을 먼저 실행해주세요.")
    tags_for_ensemble = [] # 빈 리스트로 설정하여 아래 함수 호출 건너뛰기
else:
    tags_for_ensemble = ensemble_cand_5
    print(f"▶️ 테스트셋 앙상블 시작 (대상 조합 수: {len(tags_for_ensemble)})")


if tags_for_ensemble:
    # build_submission_from_tests 함수 호출
    # prefer 인수는 동률 발생 시 선호하는 모델의 태그를 지정합니다.
    # 파일럿 300 결과에서 가장 좋았던 Qwen3 설정 중 하나를 선호 모델로 지정합니다.
    preferred_tag = 'qwen3_8b__sz672__ps0__rev1'
    if preferred_tag not in tags_for_ensemble:
        print(f"⚠️ 선호 모델 태그 ({preferred_tag})가 앙상블 조합에 없습니다. 조합의 첫 번째 모델을 선호합니다.")
        preferred_tag = tags_for_ensemble[0]


    final_submission_path = build_submission_from_tests(
        tags_for_ensemble,
        sample_csv=TEST_CSV,
        out_name=f"submission_ensemble_cand5_{datetime.now().strftime('%m%d_%H%M')}.csv",
        prefer=preferred_tag
    )
    print(f"\n✅ 최종 제출 파일 생성 완료: {final_submission_path}")
else:
    print("\n⚠️ 앙상블 대상 조합이 없어 제출 파일을 생성하지 않았습니다.")

▶️ 테스트셋 앙상블 시작 (대상 조합 수: 6)
📁 최종 제출 파일: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_ensemble_cand5_1026_1220.csv

✅ 최종 제출 파일 생성 완료: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_ensemble_cand5_1026_1220.csv


### 12-2) 테스트셋 앙상블 & 제출 파일 생성 (선택 조합)

In [62]:
# ensemble_cand_3 조합으로 테스트셋 앙상블 및 제출 파일 생성
if 'ensemble_cand_3' not in locals():
    print("⚠️ ensemble_cand_3 변수가 정의되지 않았습니다. 셀 10-1)을 먼저 실행해주세요.")
else:
    tags_for_ensemble_3 = ensemble_cand_3
    print(f"▶️ 테스트셋 앙상블 시작 (조합: ensemble_cand_3, 모델 수: {len(tags_for_ensemble_3)})")

    # 파일럿 300 결과에서 가장 좋았던 Qwen3 설정 중 하나를 선호 모델로 지정
    preferred_tag_3 = 'qwen3_8b__sz672__ps0__rev1'
    if preferred_tag_3 not in tags_for_ensemble_3:
         print(f"⚠️ 선호 모델 태그 ({preferred_tag_3})가 ensemble_cand_3 조합에 없습니다. 조합의 첫 번째 모델을 선호합니다.")
         preferred_tag_3 = tags_for_ensemble_3[0]

    final_submission_path_3 = build_submission_from_tests(
        tags_for_ensemble_3,
        sample_csv=TEST_CSV,
        out_name=f"submission_ensemble_cand3_{datetime.now().strftime('%m%d_%H%M')}.csv",
        prefer=preferred_tag_3
    )
    print(f"✅ 최종 제출 파일 생성 완료: {final_submission_path_3}")


print("-" * 50) # 구분선


# ensemble_cand_6_chatgpt_recommend 조합으로 테스트셋 앙상블 및 제출 파일 생성
if 'ensemble_cand_6_chatgpt_recommend' not in locals():
    print("⚠️ ensemble_cand_6_chatgpt_recommend 변수가 정의되지 않았습니다. 셀 10-1)을 먼저 실행해주세요.")
else:
    tags_for_ensemble_6 = ensemble_cand_6_chatgpt_recommend
    print(f"▶️ 테스트셋 앙상블 시작 (조합: ensemble_cand_6_chatgpt_recommend, 모델 수: {len(tags_for_ensemble_6)})")

    # 파일럿 300 결과에서 가장 좋았던 Qwen3 설정 중 하나를 선호 모델로 지정
    preferred_tag_6 = 'qwen3_8b__sz672__ps4__rev1' # 해당 조합에 포함된 Qwen3 최고 성능 태그
    if preferred_tag_6 not in tags_for_ensemble_6:
         print(f"⚠️ 선호 모델 태그 ({preferred_tag_6})가 ensemble_cand_6_chatgpt_recommend 조합에 없습니다. 조합의 첫 번째 모델을 선호합니다.")
         preferred_tag_6 = tags_for_ensemble_6[0]


    final_submission_path_6 = build_submission_from_tests(
        tags_for_ensemble_6,
        sample_csv=TEST_CSV,
        out_name=f"submission_ensemble_cand6_{datetime.now().strftime('%m%d_%H%M')}.csv",
        prefer=preferred_tag_6
    )
    print(f"✅ 최종 제출 파일 생성 완료: {final_submission_path_6}")

▶️ 테스트셋 앙상블 시작 (조합: ensemble_cand_3, 모델 수: 3)
📁 최종 제출 파일: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_ensemble_cand3_1026_1229.csv
✅ 최종 제출 파일 생성 완료: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_ensemble_cand3_1026_1229.csv
--------------------------------------------------
▶️ 테스트셋 앙상블 시작 (조합: ensemble_cand_6_chatgpt_recommend, 모델 수: 3)
📁 최종 제출 파일: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_ensemble_cand6_1026_1229.csv
✅ 최종 제출 파일 생성 완료: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_ensemble_cand6_1026_1229.csv


### 소프트 보팅 (Soft Voting)에 대한 설명

소프트 보팅은 앙상블 기법 중 하나로, 각 개별 모델의 **예측 확률**을 사용하여 최종 결정을 내리는 방식입니다. 단순히 가장 많은 모델이 예측한 클래스를 선택하는 하드 보팅(다수결)과 달리, 모델이 각 클래스에 대해 얼마나 '확신하는지'를 정량적으로 고려합니다.

**작동 방식:**

1.  각 모델은 입력에 대해 각 출력 클래스(여기서는 a, b, c, d)에 대한 확률 값을 계산합니다.
2.  앙상블 단계에서는 모든 모델의 예측 확률을 합산하거나 평균 냅니다 (모델별 가중치를 적용할 수도 있습니다).
3.  합산/평균된 확률이 가장 높은 클래스를 최종 예측으로 선택합니다.

**장점:**

*   모델의 예측 '강도' 또는 '확신도'를 활용하여 더 미묘한 결정을 내릴 수 있습니다.
*   단일 모델이 불확실해하는 상황에서 다른 모델의 강한 예측이 최종 결정에 더 큰 영향을 미칠 수 있습니다.
*   일반적으로 하드 보팅보다 성능이 우수한 경우가 많습니다.

**현재 코드에서 소프트 보팅 구현을 위한 고려사항:**

현재 `quick_eval` 및 `infer_test_and_save` 함수는 모델 출력 텍스트에서 최종 예측 클래스(a, b, c, d)만 파싱하고 있습니다. 모델의 실제 예측 확률 값은 저장되지 않습니다.

순수한 소프트 보팅을 구현하려면 모델의 `generate` 또는 `forward` 호출 결과에서 예측 로짓(logits) 또는 확률 분포를 얻어와 `.csv` 파일 등에 함께 저장하도록 추론 코드를 수정해야 합니다. 이후 앙상블 코드에서 이 저장된 확률 값을 사용하여 최종 예측을 계산해야 합니다.

### 13) 소프트 보팅용 추론 함수 (확률 저장)

In [63]:
import torch.nn.functional as F

def infer_test_and_save_with_probs(model, processor, test_csv, image_size, prompt_style, tag):
    t0_tot = time.time()
    test_df = pd.read_csv(test_csv)
    preds, probs_a, probs_b, probs_c, probs_d = [], [], [], [], []

    # 'a', 'b', 'c', 'd' 토큰 ID 미리 찾기
    # 모델/프로세서에 따라 토큰화 방식이 다를 수 있으므로, 가장 기본적인 형태를 시도합니다.
    # 필요한 경우 이 부분을 모델별로 더 정교하게 수정해야 할 수 있습니다.
    token_ids = {}
    for char in ['a', 'b', 'c', 'd']:
         # 여러 가능성을 고려하여 토큰 ID를 찾습니다 (예: 'a', ' a', 'A', ' A')
        found_id = None
        for text_variant in [char, f' {char}', char.upper(), f' {char.upper()}']:
            tokenized = processor.tokenizer.encode(text_variant, add_special_tokens=False)
            if len(tokenized) == 1:
                found_id = tokenized[0]
                break # 1개 토큰으로 정확히 매칭되는 경우 우선 사용
            elif len(tokenized) > 0 and found_id is None:
                found_id = tokenized[0] # 1개 토큰 매칭 없으면 첫 토큰 사용
        if found_id is not None:
            token_ids[char] = found_id
        else:
            print(f"⚠️ 경고: '{char}'에 해당하는 토큰 ID를 찾을 수 없습니다. 확률 계산에 문제가 발생할 수 있습니다.")
            token_ids[char] = -1 # 찾지 못한 경우 -1 등으로 표시

    print(f"찾은 토큰 ID: {token_ids}")


    for i in tqdm(range(len(test_df)), desc=f"test {tag} (with probs)", unit="sample"):
        row = test_df.iloc[i]
        img = load_image_resized(row[IMG_COL], image_size)
        user_text = build_prompt(row[Q_COL], row[OPT_COLS["a"]], row[OPT_COLS["b"]],
                                 row[OPT_COLS["c"]], row[OPT_COLS["d"]], prompt_style)

        # Qwen-VL 모델은 텍스트 내에 이미지 플레이스홀더가 필요할 수 있습니다.
        # build_prompt 함수가 이를 포함하지 않는다면, 여기서 수동으로 추가해야 할 수 있습니다.
        # 예: user_text = f'<img>{user_text}' # Qwen-VL 이미지 플레이스홀더 예시

        messages = [
            {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
            {"role": "user",   "content": [
                {"type": "image", "image": img},
                {"type": "text",  "text": user_text},
            ]},
        ]

        # processor.apply_chat_template를 사용할 경우
        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}


        with torch.no_grad():
            # 확률(logits)을 반환하도록 generate 호출
            out = model.generate(
                **inputs,
                max_new_tokens=1, # 다음 토큰 1개만 예측하여 확률 계산
                do_sample=False,
                temperature=1.0, # 확률 계산을 위해 temperature를 1.0으로 설정
                return_dict_in_generate=True,
                output_logits=True,
                eos_token_id=processor.tokenizer.eos_token_id,
            )

        # 예측된 첫 번째 토큰의 로짓에서 확률 추출
        # out.logits는 튜플이며, 각 요소는 (배치 크기, 시퀀스 길이, 어휘 크기) 형태입니다.
        # 우리는 첫 번째 생성 토큰에 대한 로짓이 필요합니다.
        if out.logits is not None and len(out.logits) > 0:
            logits = out.logits[0][0] # 첫 번째 샘플, 첫 번째 생성 토큰의 로짓
            probs = F.softmax(logits, dim=-1) # 로짓을 확률로 변환

            # 각 선택지 토큰 ID에 해당하는 확률 값 가져오기
            prob_a = probs[token_ids.get('a', -1)].item() if token_ids.get('a', -1) != -1 else 0.0
            prob_b = probs[token_ids.get('b', -1)].item() if token_ids.get('b', -1) != -1 else 0.0
            prob_c = probs[token_ids.get('c', -1)].item() if token_ids.get('c', -1) != -1 else 0.0
            prob_d = probs[token_ids.get('d', -1)].item() if token_ids.get('d', -1) != -1 else 0.0

            # 확률의 합이 1이 되도록 정규화 (모델이 'a','b','c','d' 외 다른 토큰을 예측할 수도 있으므로)
            total_prob_abcd = prob_a + prob_b + prob_c + prob_d
            if total_prob_abcd > 0:
                prob_a /= total_prob_abcd
                prob_b /= total_prob_abcd
                prob_c /= total_prob_abcd
                prob_d /= total_prob_abcd
            else: # a,b,c,d 토큰의 확률 합이 0인 경우 (모델이 전혀 다른 토큰 예측)
                # 이 경우 가장 높은 확률을 가진 토큰의 디코딩 결과를 사용하거나,
                # 아니면 확률을 0으로 두고 최종 앙상블에서 해당 모델 제외 등을 고려할 수 있습니다.
                # 여기서는 확률을 0으로 둡니다.
                pass # prob_a,b,c,d는 이미 0으로 초기화됨


            probs_a.append(round(prob_a, 4))
            probs_b.append(round(prob_b, 4))
            probs_c.append(round(prob_c, 4))
            probs_d.append(round(prob_d, 4))

            # 최종 예측은 여전히 하드 예측으로 저장 (확률과 함께 검토용)
            # generate max_new_tokens=1으로 실행했으므로 디코딩 결과가 짧을 것입니다.
            text = processor.batch_decode(out.sequences, skip_special_tokens=True)[0]
            preds.append(extract_choice(text)) # extract_choice를 사용하여 파싱
        else:
            # 로짓을 얻지 못한 경우 (예: 생성 실패 등)
            preds.append(None)
            probs_a.append(0.0)
            probs_b.append(0.0)
            probs_c.append(0.0)
            probs_d.append(0.0)
            print(f"⚠️ 경고: 샘플 {row[ID_COL]} 에 대해 로짓을 얻지 못했습니다.")


    df_out = pd.DataFrame({
        "id": test_df[ID_COL],
        "pred": preds, # 하드 예측 결과 (참고용)
        "prob_a": probs_a,
        "prob_b": probs_b,
        "prob_c": probs_c,
        "prob_d": probs_d,
        "tag": tag
    })
    out_path = os.path.join(EXP_ROOT,"preds", f"test_probs__{tag}.csv") # 파일 이름 변경
    df_out.to_csv(out_path, index=False)
    print("📁 저장:", out_path, "| 총소요(s):", round(time.time()-t0_tot,2))

### 14) 테스트셋 단일 추론 저장 (소프트 보팅용, 상위 6개 조합)

In [65]:
# 파일럿 300 샘플 결과 기준 상위 6개 조합 선택
if 'cmp_df_300' not in locals():
    print("⚠️ cmp_df_300 변수가 로드되지 않았습니다. 셀 8-1)을 먼저 실행하여 결과를 생성해주세요.")
    top_6_tags_300 = []
else:
    # 정확도 기준 상위 6개 조합 태그 선택
    top_6_tags_300 = cmp_df_300.head(6)["tag"].tolist()
    print(f"▶️ 파일럿 300 기준 상위 6개 조합 선택: {top_6_tags_300}")


if top_6_tags_300:
    print(f"▶️ 테스트셋 단일 추론 시작 (소프트 보팅용, 대상 조합 수: {len(top_6_tags_300)})")

    for tag_to_infer in top_6_tags_300:
        out_path = os.path.join(EXP_ROOT,"preds", f"test_probs__{tag_to_infer}.csv")

        if os.path.exists(out_path):
            print(f"✅ 파일 존재: {out_path} -> 추론 건너뛰기")
            continue

        print(f"\n🚀 테스트셋 추론 (소프트 보팅용): {tag_to_infer}")

        # tag에서 모델 ID, 해상도, 프롬프트 스타일 추출
        parts = tag_to_infer.split("__")
        short_name = parts[0]
        sz = int(parts[1].replace("sz", ""))
        st = int(parts[2].replace("ps", ""))

        # short 이름에 해당하는 model_id 찾기
        model_id = next((m_id for m_id, sh in CAND_MODELS if sh == short_name), None)

        if model_id:
            # 모델 로드
            model, proc = load_qwen_vl(model_id)

            # infer_test_and_save_with_probs 함수 호출
            infer_test_and_save_with_probs(model, proc, TEST_CSV,
                                        image_size=sz, prompt_style=st,
                                        tag=tag_to_infer)

            # 메모리 해제
            del model
            torch.cuda.empty_cache()
        else:
            print(f"⚠️ short 이름 {short_name}에 해당하는 모델 ID 찾을 수 없음. 추론 건너뛰기.")

    print("\n✅ 테스트셋 단일 추론 과정 완료 (소프트 보팅용).")
else:
    print("\n⚠️ 파일럿 300 기준 상위 조합이 없어 테스트셋 추론을 진행하지 않습니다.")

▶️ 파일럿 300 기준 상위 6개 조합 선택: ['qwen3_8b__sz672__ps0__rev1', 'qwen3_8b__sz672__ps2__rev1', 'qwen3_8b__sz672__ps4__rev1', 'qwen2_7b__sz768__ps2__rev1', 'qwen2_7b__sz672__ps3__rev1', 'qwen2_7b__sz672__ps4__rev1']
▶️ 테스트셋 단일 추론 시작 (소프트 보팅용, 대상 조합 수: 6)

🚀 테스트셋 추론 (소프트 보팅용): qwen3_8b__sz672__ps0__rev1
Loading: Qwen/Qwen3-VL-8B-Instruct


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

찾은 토큰 ID: {'a': 64, 'b': 65, 'c': 66, 'd': 67}


test qwen3_8b__sz672__ps0__rev1 (with probs): 100%|██████████| 3887/3887 [08:35<00:00,  7.54sample/s]


...

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

찾은 토큰 ID: {'a': 64, 'b': 65, 'c': 66, 'd': 67}


test qwen2_7b__sz672__ps4__rev1 (with probs): 100%|██████████| 3887/3887 [08:58<00:00,  7.22sample/s]

📁 저장: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test_probs__qwen2_7b__sz672__ps4__rev1.csv | 총소요(s): 538.2

✅ 테스트셋 단일 추론 과정 완료 (소프트 보팅용).


### 과신하는 모델의 폭주 막기 => 로짓 앙상블
- 저장된 확률 파일을 다시 로짓으로 변환하기

### 15) 확률 값을 로그 확률 (Logits 유사값)으로 변환하여 저장

모델이 예측한 확률 값에서 '과신'의 영향을 줄이기 위해, 소프트맥스 전 단계인 로짓에 가깝게 변환하여 앙상블에 활용할 수 있습니다. 간단하게 확률 값에 로그(log)를 취하여 로그 확률 값을 계산하고 저장합니다.

In [66]:
import numpy as np
import os
import pandas as pd

def convert_probs_to_logprobs(probs_df):
    """
    확률 DataFrame에서 확률 값을 로그 확률 값으로 변환합니다.
    p가 0인 경우 -inf가 될 수 있으므로 작은 epsilon 값을 더해줍니다.
    """
    epsilon = 1e-9
    logprobs_df = probs_df.copy()
    for choice in ['a', 'b', 'c', 'd']:
        prob_col = f'prob_{choice}'
        logprob_col = f'logprob_{choice}'
        # 0에 가까운 확률에 epsilon을 더하여 log(0) 방지
        logprobs_df[logprob_col] = np.log(logprobs_df[prob_col].clip(lower=epsilon))
        # 원본 확률 열은 제거하거나 필요에 따라 유지
        # del logprobs_df[prob_col]
    return logprobs_df

# 파일럿 300 결과 기준 상위 6개 조합 태그 가져오기
if 'cmp_df_300' not in locals():
    print("⚠️ cmp_df_300 변수가 로드되지 않았습니다. 셀 8-1)을 먼저 실행하여 결과를 생성해주세요.")
    tags_to_convert = []
else:
    tags_to_convert = cmp_df_300.head(6)["tag"].tolist()
    print(f"▶️ 파일럿 300 기준 상위 6개 조합 선택 (로그 확률 변환 대상): {tags_to_convert}")

if tags_to_convert:
    print(f"▶️ 확률 파일 로그 확률 변환 시작 (대상 파일 수: {len(tags_to_convert)})")

    for tag in tags_to_convert:
        probs_file_path = os.path.join(EXP_ROOT, "preds", f"test_probs__{tag}.csv")
        logprobs_file_path = os.path.join(EXP_ROOT, "preds", f"test_logprobs__{tag}.csv")

        if not os.path.exists(probs_file_path):
            print(f"⚠️ 확률 파일 없음: {probs_file_path} -> 변환 건너뛰기")
            continue

        if os.path.exists(logprobs_file_path):
            print(f"✅ 로그 확률 파일 존재: {logprobs_file_path} -> 변환 건너뛰기")
            continue

        print(f"\n🔄 확률 파일 로짓 변환: {probs_file_path}")

        # 확률 파일 로드
        probs_df = pd.read_csv(probs_file_path)

        # 로그 확률 변환
        logprobs_df = convert_probs_to_logprobs(probs_df)

        # id, pred, tag 열은 유지하고 prob_a/b/c/d 대신 logprob_a/b/c/d 저장
        cols_to_save = ['id', 'pred', 'tag'] + [f'logprob_{c}' for c in ['a', 'b', 'c', 'd']]
        logprobs_df[cols_to_save].to_csv(logprobs_file_path, index=False)

        print(f"📁 로그 확률 파일 저장 완료: {logprobs_file_path}")

    print("\n✅ 확률 파일 로그 확률 변환 과정 완료.")
else:
    print("\n⚠️ 변환할 대상 조합이 없어 로그 확률 변환을 진행하지 않습니다.")

▶️ 파일럿 300 기준 상위 6개 조합 선택 (로그 확률 변환 대상): ['qwen3_8b__sz672__ps0__rev1', 'qwen3_8b__sz672__ps2__rev1', 'qwen3_8b__sz672__ps4__rev1', 'qwen2_7b__sz768__ps2__rev1', 'qwen2_7b__sz672__ps3__rev1', 'qwen2_7b__sz672__ps4__rev1']
▶️ 확률 파일 로그 확률 변환 시작 (대상 파일 수: 6)

🔄 확률 파일 로짓 변환: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test_probs__qwen3_8b__sz672__ps0__rev1.csv
📁 로그 확률 파일 저장 완료: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test_logprobs__qwen3_8b__sz672__ps0__rev1.csv

🔄 확률 파일 로짓 변환: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test_probs__qwen3_8b__sz672__ps2__rev1.csv
📁 로그 확률 파일 저장 완료: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test_logprobs__qwen3_8b__sz672__ps2__rev1.csv

🔄 확률 파일 로짓 변환: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/test_probs__qwen3_8b__sz672__ps4__rev1.csv
📁 로그 확률 파일 저장 완료: /content/drive/My Drive/aichallenge/experiments/20251026_080951/preds/t

### 16) 테스트셋 로그 확률 기반 소프트 보팅 앙상블 함수

In [67]:
import pandas as pd
import numpy as np
import os

def build_submission_from_logprobs(tags, sample_csv=TEST_CSV, out_name=None):
    """
    로그 확률 파일들을 사용하여 소프트 보팅 앙상블을 수행하고 제출 파일을 생성합니다.
    """
    test_df = pd.read_csv(sample_csv)
    base = pd.DataFrame({"id": test_df[ID_COL]})

    logprob_cols = [f'logprob_{c}' for c in ['a', 'b', 'c', 'd']]

    # id 기준 머지
    merged_logprobs = None
    for t in tags:
        logprobs_file_path = os.path.join(EXP_ROOT, "preds", f"test_logprobs__{t}.csv")
        if not os.path.exists(logprobs_file_path):
             print(f"⚠️ 로그 확률 파일 없음: {logprobs_file_path}. 이 태그는 앙상블에서 제외됩니다.")
             continue

        df = pd.read_csv(logprobs_file_path)[['id'] + logprob_cols]
        df = df.rename(columns={c: f"{c}_{t}" for c in logprob_cols}) # 충돌 방지 위해 컬럼 이름 변경
        merged_logprobs = df if merged_logprobs is None else merged_logprobs.merge(df, on="id", how="inner")

    if merged_logprobs is None or merged_logprobs.empty:
        print("⚠️ 앙상블에 사용할 유효한 로그 확률 파일이 없습니다.")
        return None

    finals = []
    # 각 샘플에 대해 앙상블 예측 수행
    for _, row in merged_logprobs.iterrows():
        # 각 선택지별 로그 확률 합산
        summed_logprobs = {}
        for choice in ['a', 'b', 'c', 'd']:
            sum_logprob = 0
            for t in tags:
                 col_name = f'logprob_{choice}_{t}'
                 if col_name in row: # 해당 태그의 데이터가 머지되었는지 확인
                    sum_logprob += row[col_name]
            summed_logprobs[choice] = sum_logprob

        # 합산된 로그 확률이 가장 높은 선택지 선택
        final_pred = max(summed_logprobs, key=summed_logprobs.get)
        finals.append(final_pred)

    # 제출 파일 생성
    sub = pd.DataFrame({"id": merged_logprobs["id"], "answer": finals})
    tag_hash = "_".join(tags)[:50] # 파일명 너무 길어지지 않게 적당히 자름
    out = out_name or f"submission_logprob_ensemble_{tag_hash}_{datetime.now().strftime('%m%d_%H%M')}.csv"
    out_path = os.path.join(EXP_ROOT,"submissions", out)
    sub.to_csv(out_path, index=False)
    print("📁 최종 제출 파일 (로그 확률 앙상블):", out_path)
    return out_path

### 18) 테스트셋 로그 확률 기반 소프트 보팅 앙상블 실행 (A 조합)

In [68]:
# A 조합 태그 정의
ensemble_tags_A = [
    'qwen3_8b__sz672__ps0__rev1',
    'qwen3_8b__sz672__ps2__rev1',
    'qwen2_7b__sz768__ps2__rev1'
]

print(f"▶️ 테스트셋 로그 확률 기반 앙상블 시작 (조합: A 조합, 대상 조합 수: {len(ensemble_tags_A)})")

# build_submission_from_logprobs 함수 호출
final_submission_path_A = build_submission_from_logprobs(
    ensemble_tags_A,
    sample_csv=TEST_CSV,
    out_name=f"submission_logprob_ensemble_A_{datetime.now().strftime('%m%d_%H%M')}.csv",
)

if final_submission_path_A:
    print(f"\n✅ 최종 제출 파일 생성 완료: {final_submission_path_A}")
else:
    print("\n⚠️ 앙상블 대상 조합이 없어 제출 파일을 생성하지 않습니다.")

▶️ 테스트셋 로그 확률 기반 앙상블 시작 (조합: A 조합, 대상 조합 수: 3)
📁 최종 제출 파일 (로그 확률 앙상블): /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_logprob_ensemble_A_1026_1359.csv

✅ 최종 제출 파일 생성 완료: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_logprob_ensemble_A_1026_1359.csv


### 19) 테스트셋 로그 확률 기반 소프트 보팅 앙상블 실행 (B 조합)

In [69]:
# B 조합 태그 정의
ensemble_tags_B = [
    'qwen3_8b__sz672__ps0__rev1',
    'qwen3_8b__sz672__ps2__rev1',
    'qwen3_8b__sz672__ps4__rev1',
    'qwen2_7b__sz768__ps2__rev1'
]

print(f"▶️ 테스트셋 로그 확률 기반 앙상블 시작 (조합: B 조합, 대상 조합 수: {len(ensemble_tags_B)})")

# build_submission_from_logprobs 함수 호출
final_submission_path_B = build_submission_from_logprobs(
    ensemble_tags_B,
    sample_csv=TEST_CSV,
    out_name=f"submission_logprob_ensemble_B_{datetime.now().strftime('%m%d_%H%M')}.csv",
)

if final_submission_path_B:
    print(f"\n✅ 최종 제출 파일 생성 완료: {final_submission_path_B}")
else:
    print("\n⚠️ 앙상블 대상 조합이 없어 제출 파일을 생성하지 않습니다.")

▶️ 테스트셋 로그 확률 기반 앙상블 시작 (조합: B 조합, 대상 조합 수: 4)
📁 최종 제출 파일 (로그 확률 앙상블): /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_logprob_ensemble_B_1026_1401.csv

✅ 최종 제출 파일 생성 완료: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_logprob_ensemble_B_1026_1401.csv


### 20) 테스트셋 로그 확률 기반 소프트 보팅 앙상블 실행 (C 조합 - 상위 5개)

In [70]:
# C 조합 태그 정의: 파일럿 300 결과 기준 상위 6개 중 가장 낮은 하나 제외
if 'cmp_df_300' not in locals():
    print("⚠️ cmp_df_300 변수가 로드되지 않았습니다. 셀 8-1)을 먼저 실행하여 결과를 생성해주세요.")
    ensemble_tags_C = []
else:
    # 정확도 기준 상위 6개 조합 태그 선택 (n=300 결과 중)
    top_6_tags_300_full = cmp_df_300[cmp_df_300['n'] == 300].head(6).copy()

    if len(top_6_tags_300_full) < 6:
        print(f"⚠️ n=300 결과가 6개 미만입니다 ({len(top_6_tags_300_full)}개). 상위 {len(top_6_tags_300_full)}개 조합으로 앙상블을 진행합니다.")
        ensemble_tags_C = top_6_tags_300_full["tag"].tolist()
    else:
        # 정확도가 같을 경우 avg_latency_ms가 높은(느린) 것을 제외 (임의 기준)
        # 또는 자카드 유사도를 고려하여 제외할 수도 있으나, 여기서는 단순 정확도+지연시간 기준으로 가장 낮은 것 제외
        # 가장 낮은 정확도를 가진 행을 찾습니다.
        lowest_perf = top_6_tags_300_full.sort_values(['acc', 'avg_latency_ms'], ascending=[True, False]).iloc[0]
        tag_to_exclude = lowest_perf['tag']
        print(f"▶️ 파일럿 300 기준 상위 6개 중 제외할 조합: {tag_to_exclude}")

        ensemble_tags_C = top_6_tags_300_full[top_6_tags_300_full['tag'] != tag_to_exclude]["tag"].tolist()
        print(f"▶️ C 조합 (상위 5개): {ensemble_tags_C}")


if ensemble_tags_C:
    print(f"▶️ 테스트셋 로그 확률 기반 앙상블 시작 (조합: C 조합, 대상 조합 수: {len(ensemble_tags_C)})")

    # build_submission_from_logprobs 함수 호출
    final_submission_path_C = build_submission_from_logprobs(
        ensemble_tags_C,
        sample_csv=TEST_CSV,
        out_name=f"submission_logprob_ensemble_C_{datetime.now().strftime('%m%d_%H%M')}.csv",
    )

    if final_submission_path_C:
        print(f"\n✅ 최종 제출 파일 생성 완료: {final_submission_path_C}")
else:
    print("\n⚠️ 앙상블 대상 조합이 없어 제출 파일을 생성하지 않습니다.")

▶️ 파일럿 300 기준 상위 6개 중 제외할 조합: qwen2_7b__sz672__ps4__rev1
▶️ C 조합 (상위 5개): ['qwen3_8b__sz672__ps0__rev1', 'qwen3_8b__sz672__ps2__rev1', 'qwen3_8b__sz672__ps4__rev1', 'qwen2_7b__sz768__ps2__rev1', 'qwen2_7b__sz672__ps3__rev1']
▶️ 테스트셋 로그 확률 기반 앙상블 시작 (조합: C 조합, 대상 조합 수: 5)
📁 최종 제출 파일 (로그 확률 앙상블): /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_logprob_ensemble_C_1026_1401.csv

✅ 최종 제출 파일 생성 완료: /content/drive/My Drive/aichallenge/experiments/20251026_080951/submissions/submission_logprob_ensemble_C_1026_1401.csv
